# FASTGenomics Scanpy + R Analysis

<font color="red">You might want to describe your analysis briefly here, if you are planning to share it.</font>

In [1]:
# Place all your Python imports here.
import logging
import fgread
import scanpy as sc

# do not delete these imports as they are required for R support
import rpy2.rinterface_lib.callbacks
from rpy2.robjects import pandas2ri
import anndata2ri
%load_ext rpy2.ipython

In [2]:
%%R
# Place all your R library imports here
suppressPackageStartupMessages({
    library(scran)
})

In [3]:
# Place all your parameter values here.

sc.settings.verbosity = 1                                    # scanpy verbosity: errors (0), warnings (1), info (2), hints (3)
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR) # Ignore R warning messages

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()

## Raw Data
First, the raw dataset(s) will be read into an AnnData object(s).

<font color="red">You can describe your data here using markdown or delete this text.</font>

In [4]:
# Print metadata of all attached datasets
ds_info = fgread.ds_info()
ds_info

,title,id,format,organism,tissue,numberOfCells,numberOfGenes,path,file,citation,description,license,preprocessing,technology,webLink
0,"Xia et al., PNAS (2019)",dataset-022cb7ceda2840a0a4ca1d71a6fe632c,AnnData,Human,Bone,645.0,12903.0,/fastgenomics/data/dataset_0001,Xia2019_merfish_0622.h5ad,"<p>Xia, Chenglong, et al. ""Spatial transcripto...",<h3>Spatial transcriptome profiling by MERFISH...,None,None,None,https://www.pnas.org/content/116/39/19490
1,"Wu B et. al, Stem cell reports (2017)",dataset-07e9d981aed14018891984197783544a,tab-separated text,Mouse (Mus musculus),uterus,714.0,12479.0,/fastgenomics/data/dataset_0002,GSE98451_uterus_single_cell_RNA-Seq_counts.txt,"Wu, B, et. al, Reconstructing Lineage Hierarch...",<p>This dataset comprises transcriptomes from ...,None,None,CEL-seq/CEL-seq2,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...
2,3k PBMCs from a Healthy Donor,dataset-0e6e4441afac43f7988bb399ef0c3fa3,Loom,Human,peripheral blood mononuclear cells,2700.0,13714.0,/fastgenomics/data/dataset_0003,data.loom,None,<p>3k PBMCs from a Healthy Donor<br />Single C...,"<a href=""https://creativecommons.org/licenses/...",The original data was downloaded from the 10x ...,10x Chromium Single Cell 3',https://support.10xgenomics.com/single-cell-ge...


In [5]:
# Read all attached datasets into a dictionary of AnnData objects
# If you have multiple datasets they will be all loaded into RAM.
data_all = dict()
for i,ds in ds_info.iterrows():
    title = ds['title']
    data_all[title] = fgread.load_data(title)
data_all

Loading dataset "Xia et al., PNAS (2019)" in format "AnnData" from directory "/fastgenomics/data/dataset_0001"...

Loaded dataset "Xia et al., PNAS (2019)" with 645 cells and 12903 genes.

Loading dataset "Wu B et. al, Stem cell reports (2017)" in format "tab-separated text" from directory "/fastgenomics/data/dataset_0002"...

Loaded dataset "Wu B et. al, Stem cell reports (2017)" with 714 cells and 12479 genes.

Loading dataset "3k PBMCs from a Healthy Donor" in format "Loom" from directory "/fastgenomics/data/dataset_0003"...

Loaded dataset "3k PBMCs from a Healthy Donor" with 2700 cells and 32738 genes.



{'Xia et al., PNAS (2019)': AnnData object with n_obs × n_vars = 645 × 12903 
     obs: 'fg_id'
     uns: 'ds_metadata'
     obsm: 'X_spatial',
 'Wu B et. al, Stem cell reports (2017)': AnnData object with n_obs × n_vars = 714 × 12479 
     obs: 'sample', 'fg_id'
     uns: 'ds_metadata',
 '3k PBMCs from a Healthy Donor': AnnData object with n_obs × n_vars = 2700 × 32738 
     obs: 'fg_id'
     var: 'gene_ids'
     uns: 'ds_metadata'}

In [7]:
# If you only have one dataset in your analysis you can load it by
#adata = fgread.load_data()
# You can also load a specific dataset by providing its ID or title as a parameter

## Preprocessing
<font color="red">You can describe your preprocessing here or delete this text.</font>

If this is your first analysis, you might want to have a look at our tutorials on
<a href="https://beta.fastgenomics.org/webclient/ui/#/analyses/detail-analysis-42e3571101184ed78856cbd83adfeb7e" target="_blank">Getting Started with FASTGenomics Lab</a>,
the data loading (<a href="https://beta.fastgenomics.org/webclient/ui/#/analyses/detail-analysis-9975f546d76341ea85a6c5b0870654a1" target="_blank">How to Load Data in FASTGenomics (Python)</a>),
Scanpy with R support (<a href="https://beta.fastgenomics.org/webclient/ui/#/analyses/detail-analysis-2bd51db2cb564ca7bc8dc3a9caa6b09b" target="_blank">Advanced Scanpy with R Support (rpy2)</a>), or the
<a href="https://beta.fastgenomics.org/webclient/ui/#/analyses/detail-analysis-4550a1215f5747f88ac0f83ab96d2114" target="_blank">Best Practices Preprocessing Notebook</a>.

In [12]:
# This is an example of how to prepare AnnData matrices for processing with R
data = data_all[title].X.T.todense()

In [13]:
%%R -i data -o clusters
# You can use R code in cells with specified inputs and outputs
clusters <- quickCluster(data)

In [14]:
# The outputs are then available in Python
clusters

[2, 6, 4, 1, 3, ..., 1, 6, 6, 6, 4]
Length: 2700
Categories (6, object): [1, 2, 3, 4, 5, 6]